In [1]:
import os 


## Model

In [ ]:
from langchain.chat_models import ChatOpenAI

llm_model = ''

chat = ChatOpenAI(temperature = 0, 
                  model = llm_model)



## Prompt Template

In [3]:
from langchain.prompts import ChatPromptTemplate
#text = ''
#style = 'O mesmo estilo de fala que o batman teria, o tom de batman sério'
template_string = """
    Trauza o texto delimitado por "",
    para o estilo {style}
    "{text}", também complete a técnica secreta de acordo com o personagem 
     citado no style """

text = ''' 
    Voce nao poderia me derrotar, pois por ventura, sou deveras
    mais poderoso que ti hihhihi, agora contemple minha tecnica secreta: 
'''

style = ''' Vegetta muito furioso'''

prompt_template = ChatPromptTemplate.from_template(template_string)
formatPrompt = prompt_template.format_messages(
    style = style, 
    text = text 
)

print(formatPrompt[0].content)




    Trauza o texto delimitado por "",
    para o estilo  Vegetta muito furioso
    " 
    Voce nao poderia me derrotar, pois por ventura, sou deveras
    mais poderoso que ti hihhihi, agora contemple minha tecnica secreta: 
", também complete a técnica secreta de acordo com o personagem 
     citado no style 


In [18]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], input_types={}, partial_variables={}, template='\n    Trauza o texto delimitado por "",\n    para o estilo {style}\n    "{text}" ')

### Output Parsers

A way that let you define how python can be deal with the ouput of model. 

Example: I want that my output have a json format... returns needs to be a python dictionary 

* Gonna be create a parser to a json format, with the keys:

    * gift:
    * deleivery_days
    * prince_value

In [ ]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser 

In [9]:
# create one schema for each key in the output
# O schema description é puramente documental, ou possui 
# alguma finalidade de orientações ao modelo?

gift_schema = ResponseSchema(name = 'gift',
                             description= ("Was the item purchased as a gift for someone else?"
                                            "response True for yse, "
                                            "False if not or unknown")
                                            )


delivery_days_schema = ResponseSchema(
    name = 'delivery_days',
    description= ("How many days didi it take for the product to arrive?"
                  "If this information is not found, output -1")
    )


price_value_schema = ResponseSchema(
    name = 'price_value',
    description = 'Extract any sentences about the value or price, and out them as a comma separated Python list'
)

responseSchemas = [gift_schema, delivery_days_schema, price_value_schema]


In [ ]:
# Cria uma "tupla" com "n" elementos, um para cada schema/key definido.

output_parser = StructuredOutputParser.from_response_schemas(responseSchemas)
print(output_parser)

response_schemas=[ResponseSchema(name='gift', description='Was the item purchased as a gift for someone else?response True for yse, False if not or unknown', type='string'), ResponseSchema(name='delivery_days', description='How many days didi it take for the product to arrive?If this information is not found, output -1', type='string'), ResponseSchema(name='price_value', description='Extract any sentences about the value or price, and out them as a comma separated Python list', type='string')]


In [12]:
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased as a gift for someone else?response True for yse, False if not or unknown
	"delivery_days": string  // How many days didi it take for the product to arrive?If this information is not found, output -1
	"price_value": string  // Extract any sentences about the value or price, and out them as a comma separated Python list
}
```


In [16]:
customer_review = 'blablabla'
template_string = '''
    Im a template string with the following paramters:
    
    text: {text}

    {format_instructions}

 '''

prompt = ChatPromptTemplate.from_template(template = template_string)
messages = prompt.format_messages(
    text = customer_review,
    format_instructions=format_instructions
)

print(messages[0].content)


    Im a template string with the following paramters:

    text: blablabla

    The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased as a gift for someone else?response True for yse, False if not or unknown
	"delivery_days": string  // How many days didi it take for the product to arrive?If this information is not found, output -1
	"price_value": string  // Extract any sentences about the value or price, and out them as a comma separated Python list
}
```

 


In [ ]:
# invocar o modelo com o parser...
response = chat(messages)

In [ ]:
# apos o modelo processar o conteudo, utilizar o parser para a atividade de parsing do output
output_parsed = output_parser.parse(response.content)
